<a href="https://colab.research.google.com/github/mvslopes/Fluxo-de-Dados-SDBE/blob/main/03_Construindo_C%C3%B3digos_ETL_com_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**01- Construindo os códigos ETL com a Linguagem Python**

**Objetivo:**\
Após a modelagem do Data Warehouse, a criação das tabelas de dimensões e da tabela Fato, foi construído o processo de ETL - Extração, Transformação e Carregamento com a linguagem Python e o Apache Airflow.


###**1.1- Primeiro Bloco de Código " O processo de Imports"**

In [ ]:
# Primeiro Bloco de comandos os "Imports"
import csv
import airflow
import time
import pandas as pd
from datetime import datetime
from datetime import timedelta
from airflow import DAG
from airflow.operators.python_operator import PythonOperator
from airflow.operators.postgres_operator import PostgresOperator
from airflow.utils.dates import days_ago


**Comentando os código do Pacote imports**

* **import "CSV"**:\
Importa o módulo "csv" em Python. Nossa fonte de dados é um arquivo CSV. Portanto, foi necessário manipular esses arquivos em algum momento.

* **import "airflow"**:\
importa o pacote "airflow" em Python pois foi necessário construir toda a DAG (Directed Acyclic Graph, ou Grafo Direcionado Acíclico), que será lida pelo Apache Airflow.

* **import "time"**:\
Importa o módulo "time" em Python. fornece funcionalidades relacionadas ao tempo e à temporização.

* **import pandas as pd**:\
Importa o módulo "pandas" em Python que foi renomeado para "pd" para facilitar referências posteriores. O "pandas" é uma biblioteca amplamente usada para manipulação e análise de dados. Forneceu estruturas de dados e funções para trabalhar com dados tabulares.

**Em seguida, temos os imports específicos do Apache Airflow:**

* **from datetime import datetime e from datetime import timedelta:**\
 Importa as classes **datetime e timedelta** do módulo **datetime** para lidar com datas e horários.

* **from airflow import DAG:**\
 Importa a classe **DAG** do pacote **airflow**, que é fundamental para definir e configurar fluxos de trabalho no Apache Airflow.

* **from airflow.operators.python_operator import PythonOperator:**\
 Importa a classe **PythonOperator** do **pacote airflow.operators.python_operator**, que permite executar tarefas Python como parte de uma DAG.

* **from airflow.operators.postgres_operator import PostgresOperator:**\
 Importa a classe **PostgresOperator do pacote airflow.operators.postgres_operator**, que é usada para executar tarefas relacionadas ao PostgreSQL em uma DAG.

* **from airflow.utils.dates import days_ago:**\
 Importa a função **days_ago** do pacote **airflow.utils.dates**, que é útil para calcular datas com base na contagem regressiva de dias a partir da data atual.

###**1.2- Segundo Bloco de Código " Criação da DAG - Directed Acyclic Graphs"**
###**Tópico Argumentos**

In [ ]:
# Argumentos
default_args = {
    'owner': 'airflow',
    'start_date': datetime(2023, 1, 1),
    'depends_on_past': False,
    'retries': 1,
    'retry_delay': timedelta(minutes=5),
}


**Comentando o código do Argumentos**

**Argumentos:** representa um direcionário de argumentos

* **| 'owner': 'airflow' |**\
Indica que o próprietario ou responsável pela DAG é o Airflow.

* **| start_date': datetime(2023, 1, 1) |**\
 Indica que a data e a hora que a DAG começará a ser agendada e executada. A DAG começará a ser executada a partir do primeiro dia de janeiro de 2023. A partir desse ponto, a DAG será agendada de acordo com o cronograma especificado, que pode ser definido usando as outras configurações, como **schedule_interval.**

 A data  foi retroativa **datetime(2023, 1, 1)**, pois quando criamos uma GAG, nos não conseguimos mais modificar esta data, dessa forma é sempre bom colocarmos uma data retroativa, para que comecemos a execuação no momento que começarmos a DAG.

* **| 'depends_on_past': False' |**\
Indica que a execução atual da DAG não depende do sucesso da execução anterior. Ou seja, cada execução da DAG ocorrerá independentemente do resultado das execuções anteriores.

* **| 'retries': 1 |**\
Indica que em caso de falha na execução da tarefa, haverá 01 tentativa de retomar a tarefa. Se a tarefa falhar novamente após essa tentativa, não haverá mais retentativas, e a execução será considerada como falha permanente.

* **| 'retry_delay': timedelta(minutes=5) |**\
Indica que o intervalo de tempo que deve ser aguardado antes de tentar novamente uma tarefa que falhou. Neste caso, após uma falha, a tarefa será reagendada para uma nova tentativa após 5 minutos. Isso permite um atraso entre as tentativas para evitar sobrecarregar o sistema em caso de falhas frequentes.





###**Tópico criação da DAG**

In [ ]:
# Criar a DAG

# https://crontab.guru/
dag_log_solutions = DAG(dag_id = "logsol",
                   default_args = default_args,
                   schedule_interval = '0 0 * * *',
                   dagrun_timeout = timedelta(minutes = 60),
                   description = 'Job ETL de Carga no DW com Airflow',
                   start_date = airflow.utils.dates.days_ago(1)
)


**Comentando o código do Criar DAG**


* **|dag_log_solutions = DAG(dag_id = "logsol"|**\
Indica que a DAG está sendo nomeada como "dag_log_solutions_dsa" com o ID "logsol".

* **| default_args = default_args |**\
Indica a Definição doo argumento e o conteúdo desse argumento que foi a **lista de agumentos definida na etapa anterior.**

* **| schedule_interval = '0 0 * * *' |**
Indica a Definição do intervalo de agendamento: No caso de "schedule_interval = '0 0 * * *'", isso significa o seguinte:

 "0 0 * * *"é uma expressão cron que define a programação. Ela indica que a DAG será executada diariamente à meia-noite (00:00) todos os dias.
Portanto, a DAG será agendada para ser executada uma vez por dia, sempre à meia-noite, de acordo com esse cronograma.

* **| dagrun_timeout = timedelta(minutes = 60) |**

 Indica a Definição do tempo máximo permitido para a execução de uma instância da DAG (DAG run). Isso significa que cada execução da DAG tem um limite de tempo de 60 minutos (1 hora). Se a execução da DAG não for concluída dentro desse limite de tempo, ela será considerada falha.

* **| description = 'Job ETL de Carga no DW com Airflow' |**\
Indica a Definição de uma descrição ou um resumo do que é o trabalho (job) realizado pela DAG.

* **| start_date = airflow.utils.dates.days_ago(1) |**\
Indica que o start Date será um dia anterior a data que vamos criar no Airflow, ou seja  indica que a DAG começará a ser executada um dia atrás da data atual.
Essa configuração permite agendar a DAG para ser executada a partir de um ponto no passado, o que pode ser útil em cenários em que você deseja retroativamente executar tarefas ou processar dados a partir de uma data anterior.

**Obs:** Temos também um start_date no  default_args, irá gerar conflito? Não gerará conflito e se tivermos o argumentos em dois locais, vale por ultimo o que estiver a DAG

**Dessa forma temos a DAG criada que é o bloco principal para execuação do Apahe Airflow.**



###**1.3- Terceiro Bloco de Código " Função para Extarir um Arquivo**



```
def func_carrega_dados_clientes():

    # Obter o caminho do arquivo CSV
    csv_file_path = '/opt/airflow/dags/dados/DIM_CLIENTE.csv'

    # Inicializa o contador
    i = 0

    # Abrir o Arquivo CSV
    with open(csv_file_path, 'r') as f:

        reader = csv.DictReader(f)

        for item in reader:

            # Icrementa o contador
            i += 1
            
            # Extrai uma linha como dicionário
            dados_cli = dict(item)

            # Inserir dados na tabela PostgreSQL
            sql_query_cli = "INSERT INTO varejo.DIM_CLIENTE (%s) VALUES (%s)" % (','.join(dados_cli.keys()), ','.join([item for item in dados_cli.values()]))
    
            # Operador do Postgres com incremento no id da tarefa (para cada linha inserida)
            postgres_operator = PostgresOperator(task_id = 'carrega_dados_clientes_' + str(i),
                                                 sql = sql_query_cli,
                                                 params = (dados_cli),
                                                 postgres_conn_id = 'LOGDW',
                                                 dag = dag_log_solutions)
    
            # Executa o operador
            postgres_operator.execute()


tarefa_carrega_dados_clientes = PythonOperator(
        task_id = 'tarefa_carrega_dados_clientes',
        python_callable = func_carrega_dados_clientes,
        provide_context = True,
        dag = dag_log_solutions
    )

```

